In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Input
from tensorflow.math import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.math import confusion_matrix
from tensorflow.keras import layers

import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Modeling the System

### Data Processing

In [2]:
def rescale() -> Sequential:

    rescale_layer = Sequential(layers.Rescaling(1./255))

    return rescale_layer

### Data Augmentation

In [3]:
def data_aug() -> Sequential:

    data_augmentation = Sequential(layers.RandomFlip("horizontal"))

    return data_augmentation

### Model Architecture

In [4]:
def architecture() -> Sequential:

    num_classes = 10

    model = Sequential()

    model.add(Conv2D(32, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2))) 
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(num_classes, activation = "softmax"))

    return model

### Unifying the Model

In [5]:
def unify_model(rescale: Sequential, augmentation: Sequential, architecture: Sequential) -> Sequential:
    
    input_shape   = (28, 28, 1)
    inputs        = Input(shape = input_shape)
    rescale_layer = rescale(inputs)
    outputs       = architecture(inputs)
    model         = Model(inputs = inputs, outputs = outputs)

    return model

model = unify_model(rescale(), data_aug(), architecture())

# model.summary()

## Loading the data and compiling the model

### Traditional way

In [6]:
data = fashion_mnist.load_data()
train_data, test_data = data

train_inputs, train_targets = train_data
test_inputs, test_targets   = test_data

num_channels = 1
num_classes  = len(np.unique(train_targets))
input_shape  = train_inputs[0].shape[0], train_inputs[0].shape[1], num_channels


model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

history = model.fit(train_inputs,
                    train_targets,
                    batch_size = 128,
                    epochs = 5,
                    verbose = 1)

predictions = model.predict(test_inputs)

preds = [np.argmax(p) for p in predictions]

confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = num_classes)

fig = px.imshow(confusion, color_continuous_scale = "viridis")
fig.show()

Epoch 1/5
469/469 [==============================] - 20s 24ms/step - loss: 0.7673 - accuracy: 0.8147
Epoch 2/5
469/469 [==============================] - 11s 24ms/step - loss: 0.3177 - accuracy: 0.8832
Epoch 3/5
469/469 [==============================] - 12s 26ms/step - loss: 0.2692 - accuracy: 0.9006
Epoch 4/5
469/469 [==============================] - 12s 25ms/step - loss: 0.2368 - accuracy: 0.9116
Epoch 5/5
469/469 [==============================] - 12s 25ms/step - loss: 0.2126 - accuracy: 0.9209


NameError: name 'px' is not defined

### Using ImageDataGenerators in RAM

In [6]:
train_data, test_data = fashion_mnist.load_data()
(train_inputs, train_targets), (test_inputs, test_targets) = train_data, test_data

num_channels = 1
num_classes  = len(np.unique(train_targets))
input_shape  = train_inputs[0].shape[0], train_inputs[0].shape[1], num_channels

train_inputs = train_inputs.reshape(len(train_inputs), input_shape[0], input_shape[1], input_shape[2])
test_inputs  = test_inputs.reshape(len(test_inputs), input_shape[0], input_shape[1], input_shape[2])

datagen = ImageDataGenerator(horizontal_flip = True,
                             vertical_flip = True,
                             fill_mode = "nearest",
                             rotation_range = 20)

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

history = model.fit(datagen.flow(train_inputs, train_targets, batch_size = 32),
                    epochs = 5,
                    verbose = 1)

predictions = model.predict(test_inputs)

preds = [np.argmax(p) for p in predictions]

confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = num_classes)

fig = px.imshow(confusion, color_continuous_scale = "viridis")
fig.show()

Epoch 1/5


### Using tf.data in RAM

In [7]:
def prepare(ds: tuple, shuffle: bool = False, preprocess: bool = True, augment: bool = False):

    ds = tf.data.Dataset.from_tensor_slices(ds)
    
    batch_size = 32
    autotune = tf.data.AUTOTUNE

    rescale_layer = rescale()
    augment_layer = data_aug()
    
    if preprocess:
        ds = ds.map(lambda x, y: (rescale_layer(x), y), num_parallel_calls = autotune)
    
    if shuffle:
        ds = ds.shuffle(250)

    ds = ds.batch(batch_size)
    ds = ds.cache()
    ds = ds.prefetch(buffer_size = autotune)

    if augment:
        ds = ds.map(lambda x, y: (augment_layer(x), y), num_parallel_calls = autotune)

    return ds

In [9]:
import tensorflow as tf

tf.keras.backend.clear_session()

train_data, test_data = fashion_mnist.load_data()

(train_inputs, train_targets), (test_inputs, test_targets) = train_data, test_data

train_lenght = len(train_inputs)

val_inputs, val_targets    = train_inputs[int(train_lenght*0.8): ], train_targets[int(train_lenght*0.8): ]
train_inputs, train_targets = train_inputs[: int(train_lenght*0.8)], train_targets[: int(train_lenght*0.8)]

train_data = (train_inputs, train_targets)
val_data   = (val_inputs, val_targets)

train_data = prepare(train_data, shuffle = True, preprocess = False)
val_data   = prepare(val_data, preprocess = False)
test_data  = prepare(test_data, preprocess = False)

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

history = model.fit(train_data,
                    epochs = 5,
                    batch_size = 128,
                    verbose = 1,
                    validation_data = val_data)

loss, acc = model.evaluate(test_data)

predictions = model.predict(test_data)
preds = [np.argmax(p) for p in predictions]
confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = len(np.unique(test_targets)))
fig = px.imshow(confusion, color_continuous_scale = "viridis")
fig.show()

Epoch 1/5
1500/1500 [==============================] - 28s 18ms/step - loss: 0.3812 - accuracy: 0.8631 - val_loss: 0.3361 - val_accuracy: 0.8755
Epoch 2/5
1500/1500 [==============================] - 23s 16ms/step - loss: 0.3026 - accuracy: 0.8881 - val_loss: 0.3247 - val_accuracy: 0.8814
Epoch 3/5
1500/1500 [==============================] - 31s 21ms/step - loss: 0.2704 - accuracy: 0.8990 - val_loss: 0.3462 - val_accuracy: 0.8804
Epoch 4/5
1500/1500 [==============================] - 33s 22ms/step - loss: 0.2409 - accuracy: 0.9099 - val_loss: 0.3151 - val_accuracy: 0.8908
Epoch 5/5
313/313 [==============================] - 4s 12ms/step - loss: 0.3598 - accuracy: 0.8849


### For ImageDataGenerator reading from a directory. First we need to place our images in a folder. We will have 3 forlder: train, validation and test. Inside each folder we wll have a folder for every unique label: 0, 1, 2... 9.

In [14]:
train_data, test_data = fashion_mnist.load_data()
(train_inputs, train_targets), (test_inputs, test_targets) = train_data, test_data

validation_inputs, validation_targes = train_inputs[int(len(train_inputs)*0.8): ], test_targets[int(len(train_inputs)*0.8): ]
train_inputs, test_targets = train_inputs[: int(len(train_inputs)*0.8)], train_inputs[: int(len(train_inputs)*0.8)]

for sample, label in zip(train_inputs, train_targets):
    